# Реализуем метод predict_proba для KNN

Ниже реализован класс KNeighborsClassifier, который для поиска ближайших соседей использует sklearn.neighbors.NearestNeighbors

Требуется реализовать метод predict_proba для вычисления ответа классификатора.

In [102]:
import numpy as np

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import NearestNeighbors


class KNeighborsClassifier(BaseEstimator, ClassifierMixin):
    '''
    Класс, который позволит нам изучить KNN
    '''
    def __init__(self, n_neighbors=5, weights='uniform', 
                 metric='minkowski', p=2):
        '''
        Инициализируем KNN с несколькими стандартными параметрами
        '''
        assert weights in ('uniform', 'distance')
        
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.metric = metric
        
        self.NearestNeighbors = NearestNeighbors(
            n_neighbors = n_neighbors,
            metric = self.metric)
    
    def fit(self, X, y):
        '''
        Используем sklearn.neighbors.NearestNeighbors 
        для запоминания обучающей выборки
        и последующего поиска соседей
        '''
        self.NearestNeighbors.fit(X)
        
        self.n_classes = len(np.unique(y))
        self.y = y
        
    def predict_proba(self, X, use_first_zero_distant_sample=True):
        '''
        Чтобы реализовать этот метод, 
        изучите работу sklearn.neighbors.NearestNeighbors'''
        
        # получим здесь расстояния до соседей distances и их метки
        distances, indicses = self.NearestNeighbors.fit(X).kneighbors(X)
        #############################
        if self.weights == 'uniform':
            w = np.ones(distances.shape)
        else:
            # чтобы не делить на 0, 
            # добавим небольшую константу, например 1e-3
            w = 1/(distances + 1e-3)
        print(distances[1][0])
        # print(y)
        # реализуем вычисление предсказаний:
        # выбрав один объект, для каждого класса посчитаем
        # суммарный вес голосующих за него объектов
        # затем нормируем эти веса на их сумму
        # и вернем это как предсказание KNN

        probs=np.zeros([len(X), self.n_classes])
        for self_index, neighbor1, neighbor2, neighbor3, neighbor4 in indicses: # проходим по каждой точке и отмечаем её соседей
            # print('первый индекс', self_index)
            # print('у него класс:', y[self_index])
            # print('первый сосед для него с индексом',neighbor1, 'и классом', y[neighbor1])
            # print('первый вес:', w[self_index][0])
            # сначала посчитаем суммарный вес всех 5 соседей
            sum_w=sum(w[self_index])
            # print('сумма весов', sum_w, 'число соседей', len(w[self_index]))
            # print(probs.shape)
            # print(set(y))
            # теперь для каждего соседа узнаем его класс y[neighbor] и прибавим этот вес в нужную ячейку с классом
            probs[self_index][y[self_index]] += w[self_index][0]
            probs[self_index][y[neighbor1]] += w[self_index][1]
            probs[self_index][y[neighbor2]] += w[self_index][2]
            probs[self_index][y[neighbor3]] += w[self_index][3]
            probs[self_index][y[neighbor4]] += w[self_index][4]
            # нормализируем веса
            probs[self_index]=probs[self_index]/sum_w
        return probs

# Загрузим данные и обучим классификатор

In [103]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)

knn = KNeighborsClassifier(weights='distance')
knn.fit(X, y)
prediction = knn.predict_proba(X, y)

0.0


Поскольку мы используем одну и ту же выборку для обучения и предсказания, ближайшим соседом любого объекта будет он же сам. В качестве упражнения предлагаю реализовать метод transform, который реализует получение предсказаний для обучающей выборки, но для каждого объекта не будет учитывать его самого.

Посмотрим, в каких объектах max(prediction) != 1:

In [105]:
inds = np.arange(len(prediction))[prediction.max(1) != 1]
print(inds)

# [ 56  68  70  72  77  83 106 110 119 123 127 133 134 138 146]

[ 56  68  70  72  77  83 101 106 110 119 123 127 133 134 138 142 146]


Несколько примеров, на которых можно проверить правильность реализованного метода:

In [106]:
for i in 1, 4, -1:
    print(inds[i], prediction[inds[i]])

# 68 [0.         0.99816311 0.00183689]
# 77 [0.         0.99527902 0.00472098]
# 146 [0.         0.00239145 0.99760855]

68 [0.         0.99816311 0.00183689]
77 [0.         0.99527902 0.00472098]
146 [0.         0.00239145 0.99760855]


**Примечание:** отличие в третьем-четвертом знаке после запятой в тестах не должно повлиять на сдачу задания

# Ответы для формы

В форму требуется ввести max(prediction) для объекта. Если метод реализован верно, то ячейка ниже распечатает ответы, которые нужно ввести в форму

In [107]:
for i in 56, 83, 127:
    print('{:.2f}'.format(max(prediction[i])))

1.00
0.99
1.00
